This script checks whether there is a difference in the post-choice pupil timecourse congruency effect as a function of condition (priorOnly vs. pretoneOnly) for the section "Rule- and stimulus-based biases exhibited distinct physiological signatures"

In [1]:
#clear memory
rm(list=ls())

### this runs regressions for pretone_pLH choice-aligned pupil data

## LOADING data/libraries ##

#load libraries
library(lme4)
library(lmerTest)
library(car)
library(plyr)
library(dplyr)
library(ggplot2)
#library(afex)
library(emmeans)
emm_options(lmerTest.limit = Inf, lmer.df = "satterthwaite")


switch(Sys.info()[['sysname']],
       Windows = setwd(file.path(
         Sys.getenv('USERPROFILE'),'Dropbox/Goldlab/AuditoryPriors/data processing/pupil')),
       Darwin = setwd('~/Dropbox/Goldlab/AuditoryPriors/data processing/pupil')
)

#path to data files
switch(Sys.info()[['sysname']],
       Windows = DATA_OUT_PATH <- (paste0(
         Sys.getenv('USERPROFILE'),'/OneDrive/Goldlab/AuditoryPriors/cached data/')),
       Darwin = DATA_OUT_PATH <- '~/OneDrive/Goldlab/AuditoryPriors/cached data/'
)
pd_files= paste0(DATA_OUT_PATH,
                 c('pupil_data_ds50_forR_09-Feb-2021.csv',
                   'pupil_data_pretoneOnly_ds50_forR_05-May-2021.csv'))
baseline_files = paste0(DATA_OUT_PATH,
                        c('pupil_data_bl_forR_09-Feb-2021.csv',
                          'pupil_data_pretoneOnly_bl_forR_05-May-2021.csv'))
ID_files <- paste0(DATA_OUT_PATH,
                   c('pupil_data_ds50_id2subj_forR_09-Feb-2021.csv',
                    'pupil_data_pretoneOnly_ds50_id2subj_forR_05-May-2021.csv'))

#our ROI for looking at interaction
choice_period_ms.c <- c(0,1000) 

Loading required package: Matrix

Warning message:
"package 'lmerTest' was built under R version 4.0.4"
Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar


Attaching package: 'lmerTest'


The following object is masked from 'package:lme4':

    lmer


The following object is masked from 'package:stats':

    step


Warning message:
"package 'car' was built under R version 4.0.4"
Loading required package: carData

Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


Attaching package: 'dplyr'


The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from 'package:car':

    recode


The following objects are masked from 'package:stats'

In [2]:
#load data

ID_df <- lapply(ID_files,
                function(x) read.table(x,sep=',', header=TRUE, 
                                       stringsAsFactors=FALSE,na.strings = c('NaN')))
ID_df <- inner_join(ID_df[[1]],ID_df[[2]],by='subject')
ID_df$dataIDall <- 1:nrow(ID_df) #add global ID for both blocks

bl_df <- lapply(baseline_files,
                function(x) read.table(x,sep=',', header=TRUE, 
                                       stringsAsFactors=FALSE,na.strings = c('NaN')))
#2nd baseline was taken wrt already baselined data, so this will put it on same scale as original baseline
bl_df[[2]]$pupilBL2 <- bl_df[[2]]$pupilBL2 + bl_df[[2]]$pupilBL 

pd_df <- lapply(pd_files,
                function(x) read.table(x,sep=',', header=TRUE, 
                                       stringsAsFactors=FALSE,na.strings = c('NaN')))

In [3]:
#join new IDs
pd_df[[1]] <- inner_join(pd_df[[1]],select(ID_df,c("dataID.x","dataIDall")),
                        by=c("dataID"="dataID.x"))
pd_df[[2]] <- inner_join(pd_df[[2]],select(ID_df,c("dataID.y","dataIDall")),
                        by=c("dataID"="dataID.y"))

#add in baseline data
pd_df[[1]] <- left_join(pd_df[[1]],bl_df[[1]][,c('dataID','trialN','pupilBL')],
                        by=c('dataID','trialN'))
pd_df[[2]] <- left_join(pd_df[[2]],bl_df[[2]][,c('dataID','trialN','pupilBL2')],
                        by=c('dataID','trialN'))

In [4]:
#join pupil data

#first clean up so we have matching columns in both datasets
pd_df[[1]]$aSNR <- abs(pd_df[[1]]$SNR) #aSNR not in this file
pd_df[[1]] <- subset(pd_df[[1]],congruent!=-1) #remove no prior trials
pd_df[[1]]$pretoneLength <- 0

pd_df[[2]] <- subset(pd_df[[2]],congruent %in% c(0,3)) #only incon-incon and con-con trials
pd_df[[2]]$congruent[pd_df[[2]]$congruent==3] <- 1 #recode to match priorOnly
#used different name for the 2nd baseline cols so need to rename
pd_df[[2]]$pupilCblz <- NULL
pd_df[[2]] <- rename(pd_df[[2]],pupilBL=pupilBL2,pupilCblz=pupilCblz2) 

#add block info
pd_df[[1]]$block <- 'priorOnly'
pd_df[[2]]$block <- 'pretoneOnly'

#now join
joincols <- c('trialN','trial_time_choice','pupilCblz','posXCbl','posYCbl','pupilBL','aSNR',
    'isH','success','choice01','pretoneLength','congruent','dataIDall','block')

pd_df <- rbind(pd_df[[1]][,joincols],pd_df[[2]][,joincols])
head(pd_df)

,trialN,trial_time_choice,pupilCblz,posXCbl,posYCbl,pupilBL,aSNR,isH,success,choice01,pretoneLength,congruent,dataIDall,block
,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>
2228,49,-780,-0.2874598,10.095451,21.15839,0.441735,0.5,1,1,1,0,1,1,priorOnly
2229,49,-760,-0.2677306,9.145169,20.85463,0.441735,0.5,1,1,1,0,1,1,priorOnly
2230,49,-740,-0.2476534,9.711329,20.60854,0.441735,0.5,1,1,1,0,1,1,priorOnly
2231,49,-720,-0.2272123,9.787316,19.31012,0.441735,0.5,1,1,1,0,1,1,priorOnly
2232,49,-700,-0.2066975,9.344649,16.53824,0.441735,0.5,1,1,1,0,1,1,priorOnly
2233,49,-680,-0.1864250,7.554520,13.03064,0.441735,0.5,1,1,1,0,1,1,priorOnly


In [5]:
#set up factors
pd_df$congruent.f <- factor(pd_df$congruent,levels=c(0,1),
                            labels=c("incongruent","congruent"))
contrasts(pd_df$congruent.f) <- contr.sum(2)
contrasts(pd_df$congruent.f)

my_simple2<-contr.treatment(2,base=2) - matrix(rep(1/2,2))
pd_df$congruent.fs <- pd_df$congruent.f
contrasts(pd_df$congruent.fs) <- my_simple2
contrasts(pd_df$congruent.fs)

unique(pd_df[,c('congruent','congruent.f','congruent.fs')])

pd_df$isH.f <- factor(pd_df$isH,levels=c(1,0),
                      labels=c("high","low"))
contrasts(pd_df$isH.f) <- contr.sum(2)
contrasts(pd_df$isH.f)

pd_df$block.f <- factor(pd_df$block,levels=c('priorOnly','pretoneOnly'))
contrasts(pd_df$block.f) <- contr.sum(2)
contrasts(pd_df$block.f)

pd_df$block.fs <- pd_df$block.f
contrasts(pd_df$block.fs) <- my_simple2
contrasts(pd_df$block.fs)

unique(pd_df[,c('block','block.f','block.fs')])



#get rid of missing before scaling vars (probably already done w/ new preproc)
pd_df <- pd_df[complete.cases(pd_df),]

incongruent,1
congruent,-1


,1
incongruent,0.5
congruent,-0.5


,congruent,congruent.f,congruent.fs
,<dbl>,<fct>,<fct>
2228,1,congruent,congruent
2327,0,incongruent,incongruent


high,1
low,-1


priorOnly,1
pretoneOnly,-1


,1
priorOnly,0.5
pretoneOnly,-0.5


,block,block.f,block.fs
,<chr>,<fct>,<fct>
2228,priorOnly,priorOnly,priorOnly
1,pretoneOnly,pretoneOnly,pretoneOnly


In [6]:
#get a handle on descriptives to inform scaling
summarise(group_by(pd_df,block),bl=mean(pupilBL),bl_sd=sd(pupilBL))
scalevars_stats <- summarise(pd_df,
                             m_bl=mean(pupilBL,na.rm=T),
                             sd_bl=sd(pupilBL,na.rm=T),
                             m_x=mean(posXCbl,na.rm=T),
                             sd_x=sd(posXCbl,na.rm=T),
                             m_y=mean(posYCbl,na.rm=T),
                             sd_y=sd(posYCbl,na.rm=T),
                             m_asnr=mean(aSNR,na.rm=T),
                             sd_asnr=sd(aSNR,na.rm=T),
)
scalevars_stats

#set up variables: centering/scaling
pd_df$zaSNR <- scale(pd_df$aSNR)
pd_df$blz <- scale(pd_df$pupilBL,center=T,scale=F) #not scaling since already scaled
pd_df$posX <- scale(pd_df$posXCbl)
pd_df$posY <- scale(pd_df$posYCbl)
pd_df$zptlen <- scale(pd_df$pretoneLength)

`summarise()` ungrouping output (override with `.groups` argument)



block,bl,bl_sd
<chr>,<dbl>,<dbl>
pretoneOnly,-0.1316781,0.9668775
priorOnly,-0.1581757,0.9529235


m_bl,sd_bl,m_x,sd_x,m_y,sd_y,m_asnr,sd_asnr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.1398449,0.9626759,0.001030958,16.05118,0.1734558,19.86706,0.2054237,0.1717053


In [7]:
#we seem to have a reasonable amount of data per subject after all this wrangling...

length(unique(pd_df$dataIDall))
nrow(unique(pd_df[,c('dataIDall','block','trialN')]))
nrow(unique(pd_df[,c('dataIDall','block','trialN')]))/length(unique(pd_df$dataIDall))

[1] 40

[1] 17513

[1] 437.825

# Correct trials, precue window

In [8]:
pd_choicedf.cpc <- subset(pd_df,success==1 & 
                            trial_time_choice >= choice_period_ms.c[1] & 
                            trial_time_choice <= choice_period_ms.c[2])

pd_choicedf.cpc.ave <- summarise(group_by(pd_choicedf.cpc,
                                          dataIDall,trialN,
                                          zaSNR,congruent.f,congruent.fs,isH.f,
                                          block.f,,block.fs,zptlen),
                                 pupilCblz=mean(pupilCblz),
                                 blz=mean(blz),#blz could be a grouping factor but this is fine
                                 posX=mean(posX),
                                 posY=mean(posY))

#set up vars for zero corr
pd_choicedf.cpc.ave$isH.f1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$isH.f,pd_choicedf.cpc.ave)[,2]
pd_choicedf.cpc.ave$congruent.f1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$congruent.f,pd_choicedf.cpc.ave)[,2]
pd_choicedf.cpc.ave$block.f1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$block.f,pd_choicedf.cpc.ave)[,2]
pd_choicedf.cpc.ave$congruent.fs1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$congruent.fs,pd_choicedf.cpc.ave)[,2]
pd_choicedf.cpc.ave$block.fs1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$block.fs,pd_choicedf.cpc.ave)[,2]
unique(pd_choicedf.cpc.ave[,c("block.fs","block.fs1")])
unique(pd_choicedf.cpc.ave[,c("isH.f","isH.f1")])
unique(pd_choicedf.cpc.ave[,c("congruent.fs","congruent.fs1")])

`summarise()` regrouping output by 'dataIDall', 'trialN', 'zaSNR', 'congruent.f', 'congruent.fs', 'isH.f', 'block.f', 'block.fs' (override with `.groups` argument)



block.fs,block.fs1
<fct>,<dbl>
pretoneOnly,-0.5
priorOnly,0.5


isH.f,isH.f1
<fct>,<dbl>
low,-1
high,1


congruent.fs,congruent.fs1
<fct>,<dbl>
congruent,-0.5
incongruent,0.5


In [9]:
#let's do simple-coded congruent/block version to get a proper beta

#full model is singular, so going to zero corr model
choice.lm.cpc.fs <- lmer(pupilCblz~
                           block.fs*congruent.fs + zaSNR +
                           isH.f + blz + posX + posY + zptlen + 
                           (1 + block.fs1*congruent.fs1 + 
                              zaSNR + isH.f1 + blz + posX+ posY + zptlen|dataIDall),
                         data=pd_choicedf.cpc.ave,
                         control=lmerControl(optimizer="bobyqa",
                                             optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see ?isSingular



In [10]:
#zero corr model
choice.lm.cpc.fs.zc <- lmer(pupilCblz~
                           block.fs*congruent.fs + zaSNR +
                           isH.f + blz + posX + posY + zptlen + 
                           (1 + block.fs1*congruent.fs1 + 
                              zaSNR + isH.f1 + blz + posX+ posY + zptlen||dataIDall),
                         data=pd_choicedf.cpc.ave,
                         control=lmerControl(optimizer="bobyqa",
                                             optCtrl=list(maxfun=2e5)))

In [11]:
summary(choice.lm.cpc.fs.zc)
anova(choice.lm.cpc.fs.zc ,type="III")

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: pupilCblz ~ block.fs * congruent.fs + zaSNR + isH.f + blz + posX +  
    posY + zptlen + (1 + block.fs1 * congruent.fs1 + zaSNR +  
    isH.f1 + blz + posX + posY + zptlen || dataIDall)
   Data: pd_choicedf.cpc.ave
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

REML criterion at convergence: 8479.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.0855 -0.5520 -0.0037  0.5604  6.0220 

Random effects:
 Groups      Name                    Variance  Std.Dev.
 dataIDall   (Intercept)             0.0202213 0.14220 
 dataIDall.1 block.fs1               0.1319592 0.36326 
 dataIDall.2 congruent.fs1           0.0010054 0.03171 
 dataIDall.3 zaSNR                   0.0006180 0.02486 
 dataIDall.4 isH.f1                  0.0004990 0.02234 
 dataIDall.5 blz                     0.0104218 0.10209 
 dataIDall.6 posX                    0.0015675 0.03959 
 dataIDal

,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
block.fs,9.07266212,9.07266212,1,39.01564,85.6188905,2.190940e-11
congruent.fs,4.46461505,4.46461505,1,35.30980,42.1326599,1.682715e-07
zaSNR,2.21446403,2.21446403,1,39.59767,20.8979405,4.667883e-05
isH.f,0.04517844,0.04517844,1,36.98000,0.4263498,5.178246e-01
blz,19.20404705,19.20404705,1,38.46594,181.2289690,3.905680e-16
posX,2.04681162,2.04681162,1,37.09116,19.3158014,8.940326e-05
posY,2.11723333,2.11723333,1,38.68117,19.9803725,6.648261e-05
zptlen,19.04916453,19.04916453,1,40.75900,179.7673396,1.560497e-16
block.fs:congruent.fs,3.37960391,3.37960391,1,38.35647,31.8933884,1.684058e-06
